In [1]:
import sys
import os

# Add the parent directory of 'vegetation-period-NDVI' to the sys.path
sys.path.append(os.path.abspath(os.path.join('..')))


In [2]:
import ee
import geemap
from time_series import extract_time_ranges, get_harmonic_ts

In [3]:
ee.Initialize(project="thurgau-irrigation")

In [4]:
yr = "2018"
time_intervals = extract_time_ranges(
    time_range=[yr + "-03-01", yr + "-10-31"], agg_interval=15
)
print(time_intervals.getInfo())

[[{'type': 'Date', 'value': 1519862400000}, {'type': 'Date', 'value': 1521201600000}], [{'type': 'Date', 'value': 1521201600000}, {'type': 'Date', 'value': 1522540800000}], [{'type': 'Date', 'value': 1522540800000}, {'type': 'Date', 'value': 1523836800000}], [{'type': 'Date', 'value': 1523836800000}, {'type': 'Date', 'value': 1525132800000}], [{'type': 'Date', 'value': 1525132800000}, {'type': 'Date', 'value': 1526472000000}], [{'type': 'Date', 'value': 1526472000000}, {'type': 'Date', 'value': 1527811200000}], [{'type': 'Date', 'value': 1527811200000}, {'type': 'Date', 'value': 1529107200000}], [{'type': 'Date', 'value': 1529107200000}, {'type': 'Date', 'value': 1530403200000}], [{'type': 'Date', 'value': 1530403200000}, {'type': 'Date', 'value': 1531742400000}], [{'type': 'Date', 'value': 1531742400000}, {'type': 'Date', 'value': 1533081600000}], [{'type': 'Date', 'value': 1533081600000}, {'type': 'Date', 'value': 1534420800000}], [{'type': 'Date', 'value': 1534420800000}, {'type': '

In [5]:
test_aoi2 = ee.Geometry.Polygon(
    [
        [
            [8.737412756413452, 47.648802254332814],
            [8.737412756413452, 47.61548626028342],
            [8.829423254460327, 47.61548626028342],
            [8.829423254460327, 47.648802254332814],
        ]
    ]
)

In [6]:
filtered_sentinel_data = get_harmonic_ts(year=yr, aoi=test_aoi2, time_intervals=time_intervals)

/opt/anaconda3/lib/python3.11/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for JRC/GSW1_2/GlobalSurfaceWater! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/JRC_GSW1_2_GlobalSurfaceWater

  warnings.warn(warning, category=DeprecationWarning)


TypeError: add_cloud_shadow_mask() takes from 1 to 2 positional arguments but 3 were given